In [46]:
# 0. import & hyperparameters
import os

import torch
from torch import nn

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

learning_rate = 1e-3
batch_size = 64
epochs = 10

In [47]:
# 1. Download dataset from torchvision
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [48]:
# 2. Dataload
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [49]:
# 3. Network define
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [50]:
# 4. Loss function and optimizer define
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [51]:
# 5. Train model
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) # current batch's loss
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

PATH = './Fasion_net.pth'
torch.save(model.state_dict(), PATH)


Epoch 1
-------------------------------
loss: 2.292230  [    0/60000]
loss: 2.148890  [ 6400/60000]
loss: 1.884535  [12800/60000]
loss: 1.588665  [19200/60000]
loss: 1.239185  [25600/60000]
loss: 1.137325  [32000/60000]
loss: 0.876608  [38400/60000]
loss: 0.820988  [44800/60000]
loss: 0.810452  [51200/60000]
loss: 0.668815  [57600/60000]
Epoch 2
-------------------------------
loss: 0.742107  [    0/60000]
loss: 0.651336  [ 6400/60000]
loss: 0.621313  [12800/60000]
loss: 0.720356  [19200/60000]
loss: 0.688134  [25600/60000]
loss: 0.666678  [32000/60000]
loss: 0.515505  [38400/60000]
loss: 0.483183  [44800/60000]
loss: 0.539179  [51200/60000]
loss: 0.529213  [57600/60000]
Epoch 3
-------------------------------
loss: 0.710541  [    0/60000]
loss: 0.570037  [ 6400/60000]
loss: 0.589611  [12800/60000]
loss: 0.440941  [19200/60000]
loss: 0.536891  [25600/60000]
loss: 0.679892  [32000/60000]
loss: 0.635479  [38400/60000]
loss: 0.637921  [44800/60000]
loss: 0.488425  [51200/60000]
loss: 0.29

In [52]:
# 6. Evaluate model
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(PATH))

test_loop(test_dataloader, model, loss_fn)


Test Error: 
 Accuracy: 84.2%, Avg loss: 0.442325 

